In [1]:
import pandas as pd
import numpy as np
import math
%matplotlib inline

In [2]:
import os
path_working_directory = os.getcwd()

In [3]:
def get_dict_of_categorical_col(df, col_name):
    values = np.unique(df[col_name].values).tolist()
    values_dict = {}
    for i in range(len(values)):
        values_dict[values[i]] = i + 1
    return values_dict

def binary_codification_column(df, col, cant_cols):
    col_dict = get_dict_of_categorical_col(df, col)
    cols = [col + "_bit_" + str(i) for i in range(8)][::-1]
    def get_cols_bits(value):
        n = col_dict[value]
        n_binary = "{0:b}".format(n).zfill(cant_cols)
        tmp = [n_binary[i] for i in range(cant_cols)]
        return pd.Series(tmp,index=cols)
    df[cols] = df[col].apply(get_cols_bits)
    return df.drop(col,axis=1)

# Pre procesamiento

## Df Avisos

In [4]:
avisos_detalles = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_6_avisos_detalle.csv"), low_memory=False)
avisos_detallesE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_6_avisos_detalle.csv"), low_memory=False)
avisos_detalles15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_6_avisos_detalle.csv"), low_memory=False)
avisos_faltantes = pd.read_csv(os.path.join(path_working_directory, "fiuba_6_avisos_detalle_missing_nivel_laboral.csv"), low_memory=False)

avisos_detalles_comb = avisos_detalles.append(avisos_detalles15A)
avisos_detalles_comb = avisos_detalles_comb.append(avisos_detallesE6)
avisos_detalles_comb = avisos_detalles_comb.append(avisos_faltantes)

In [5]:
avisos_detalles_comb = avisos_detalles_comb.drop_duplicates(subset='idaviso', keep='last')

In [6]:
del avisos_detalles
del avisos_detalles15A
del avisos_detallesE6
del avisos_faltantes

In [7]:
#Dropeamos estas dos columnas dado que no aportan informacion, en la mayoria de los datos se encuentran incompletas
avisos_detalles_comb.drop('ciudad', axis=1, inplace=True)
avisos_detalles_comb.drop('mapacalle', axis=1, inplace=True)

In [8]:
from nltk import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

#Funcion para eliminar las stopwords y aquellos caracteres que no sean letras de un texto
#Devuelve una lista con todas las palabras en el texto
def tokenize(string):
    stops = stopwords.words('spanish')
    non_letters = list(punctuation)
    non_letters.extend(['¡','¿']) #Agregamos los signos de cierre que faltan en la lista
    non_letters.extend(map(str,range(10))) #Agregamos los numeros  
    text = "".join([w for w in string if w not in non_letters])
    text = filter(lambda x: len(x)>0 and x not in stops, map(lambda x: x.lower().strip(),text.split(' ')))
    return word_tokenize(" ".join(text))

#Funcion para extraer solamente las palabras de un aviso
def extract_html(string):
    string_without_html = BeautifulSoup(string,"lxml")
    return tokenize(string_without_html.text.lower())


In [9]:
avisos_detalles_comb['descripcion'] = avisos_detalles_comb.descripcion.apply(extract_html)
avisos_detalles_comb['titulo'] = avisos_detalles_comb.titulo.apply(tokenize)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  del sys.path[0]
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [10]:
avisos_detalles_comb.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
4,1000872556,1,"[operarios, planta, rubro, electrodomésticos]","[operarios, plantanos, encontramos, búsqueda, ...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
8,9240880,1,"[productores, asesores, independiente, ventas,...","[agente, oficial, selecciona, nuestra, búsqued...",Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
19,1110185164,1,"[administrativo, recepción]","[centro, médico, accord, encuentra, búsqueda, ...",Gran Buenos Aires,Full-time,Junior,Administración,Unión Personal
21,1110513885,1,"[distribuidor, domiciliario, moto, san, miguel]","[importante, correo, privado, ubicado, san, mi...",Gran Buenos Aires,Full-time,Junior,Distribución,Asoko Tempo SA
32,1111034024,1,"[vendedores, venta, medicina, pre, paga, obra,...","[comercializadora, incorpora, vendedores, vent...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,JELS SRL


In [11]:
avisos_detalles_comb.to_pickle("pickles/avisos_detalles_comb_sin_features_des_tit")

In [12]:
#definimos una lista de palabras que consideramos positivas y que pueden atraer a los usuarios
#se podria utilizar un tf-idf para determinar estas palabras
features_text = ["crecimiento", "cálido", "multinacional", "desafío", "jóven", "experimentado", "flexible", "flexibilidad"]
for i, feature in enumerate(features_text):
    avisos_detalles_comb["ft_{}".format(i)] = avisos_detalles_comb.descripcion.apply(lambda x: 1 if feature in x else 0)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  """


In [13]:
avisos_detalles_comb.to_pickle("pickles/avisos_detalles_comb_con_features_des_tit")

In [4]:
avisos = pd.read_pickle("pickles/avisos_detalles_comb_con_features_des_tit")

In [5]:
avisos = binary_codification_column(avisos, 'nombre_area', 8)

In [6]:
#Se desea realizar one hot encoding
#Debido a la escasa cantidad de datos en varias zonas, se las une para evitar
#agregar dimensiones de mas al set de datos
def reducir_nombre_zona(nombre):
    if nombre == "Gran Buenos Aires" or nombre == "GBA Oeste":
        return "Gran Buenos Aires"
    if nombre == "Buenos Aires (fuera de GBA)" or nombre == "La Plata":
        return "Buenos Aires (fuera de GBA)"
    if nombre == "Capital Federal":
        return "Capital Federal"
    return "Interior"

avisos['nombre_zona'] = avisos['nombre_zona'].apply(reducir_nombre_zona)

In [7]:
avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,denominacion_empresa,ft_0,ft_1,...,ft_6,ft_7,nombre_area_bit_7,nombre_area_bit_6,nombre_area_bit_5,nombre_area_bit_4,nombre_area_bit_3,nombre_area_bit_2,nombre_area_bit_1,nombre_area_bit_0
4,1000872556,1,"[operarios, planta, rubro, electrodomésticos]","[operarios, plantanos, encontramos, búsqueda, ...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,ELECTRO OUTLET SRL,0,0,...,0,0,1,0,0,1,0,0,0,0
8,9240880,1,"[productores, asesores, independiente, ventas,...","[agente, oficial, selecciona, nuestra, búsqued...",Capital Federal,Full-time,Jefe / Supervisor / Responsable,Agencia Oficial Alejandro Arizaga,1,0,...,0,0,0,0,0,1,1,1,1,1
19,1110185164,1,"[administrativo, recepción]","[centro, médico, accord, encuentra, búsqueda, ...",Gran Buenos Aires,Full-time,Junior,Unión Personal,0,0,...,0,0,0,0,0,0,0,0,1,1
21,1110513885,1,"[distribuidor, domiciliario, moto, san, miguel]","[importante, correo, privado, ubicado, san, mi...",Gran Buenos Aires,Full-time,Junior,Asoko Tempo SA,0,0,...,0,0,0,1,0,0,0,0,0,0
32,1111034024,1,"[vendedores, venta, medicina, pre, paga, obra,...","[comercializadora, incorpora, vendedores, vent...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,JELS SRL,0,0,...,0,0,0,0,0,1,1,1,1,1


In [8]:
avisos.to_pickle("pickles/avisos")

## Df Usuarios

In [4]:
educacion = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edad = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)

educacionE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edadE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)

educacion15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edad15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)

educacion_comb = educacion.append(educacionE6)
educacion_comb = educacion_comb.append(educacion15A)

genero_edad_comb = genero_edad.append(genero_edadE6)
genero_edad_comb = genero_edad_comb.append(genero_edad15A)

In [5]:
del educacion
del educacionE6
del educacion15A
del genero_edad
del genero_edadE6
del genero_edad15A

In [6]:
#Generamos columnas estudio en curso, estudio graduado
tipos_de_estudio = ['Ninguno', 'Otro', 'Secundario', 'Terciario/Técnico', 'Universitario', 'Master', 'Posgrado', 'Doctorado']
def get_dict_tipos_de_estudio():
    dict = {}
    for x in range(len(tipos_de_estudio)):
        dict[tipos_de_estudio[x]] = x
    return dict

dict_tipos_de_estudio = get_dict_tipos_de_estudio()

def mejor_estudio(estudio1, estudio2):
    return tipos_de_estudio[max(tipos_de_estudio.index(estudio1), tipos_de_estudio.index(estudio2))]

en_curso = educacion_comb.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'En Curso' else 0, axis=1)
graduado = educacion_comb.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'Graduado' else 0, axis=1)
abandonado = educacion_comb.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'Abandonado' else 0, axis=1)
educacion_comb2 = educacion_comb.assign(estudio_en_curso=en_curso)
educacion_comb2 = educacion_comb2.assign(estudio_graduado=graduado)
educacion_comb2 = educacion_comb2.assign(estudio_abandonado=abandonado)
del en_curso
del graduado
del abandonado

In [8]:
educacion_grupos = educacion_comb2.groupby('idpostulante')
educacion_comb = pd.DataFrame({
    'estudio_en_curso': educacion_grupos['estudio_en_curso'].max(),
    'estudio_graduado': educacion_grupos['estudio_graduado'].max(),
    'estudio_abandonado': educacion_grupos['estudio_abandonado'].max()
}).reset_index()
educacion_comb['estudio_en_curso'] = educacion_comb.apply(lambda x: x['estudio_en_curso'] if x['estudio_en_curso'] > x['estudio_graduado'] else 0, axis=1)
educacion_comb['estudio_abandonado'] = educacion_comb.apply(lambda x: x['estudio_abandonado'] if x['estudio_abandonado'] >= x['estudio_en_curso'] else 0, axis=1)
del educacion_grupos
del educacion_comb2

In [9]:
genero_edad_comb = genero_edad_comb.drop_duplicates(subset='idpostulante', keep='last')

In [10]:
len(genero_edad_comb)

504407

In [11]:
import datetime as datetime
import dateutil 

genero_edad_comb_con_fecha = genero_edad_comb[genero_edad_comb.fechanacimiento.notnull()]
genero_edad_comb_sin_fecha = genero_edad_comb[genero_edad_comb.fechanacimiento.isnull()]
# len(edades['fechanacimiento'][edades['fechanacimiento'].apply(lambda x: isinstance(x, str))])
genero_edad_comb_con_fecha['edades'] = pd.to_datetime(genero_edad_comb_con_fecha['fechanacimiento'], format='%Y-%m-%d', errors='coerce')

genero_edad_con_edad = genero_edad_comb_con_fecha[genero_edad_comb_con_fecha.edades.notnull()]
genero_edad_sin_edad = genero_edad_comb_con_fecha[genero_edad_comb_con_fecha.edades.isnull()]

total = len(genero_edad_comb_sin_fecha) + len(genero_edad_con_edad) + len(genero_edad_sin_edad)
print total

genero_edad_con_edad['edades'] = genero_edad_con_edad['edades'].apply(lambda x: int(dateutil.relativedelta.relativedelta(datetime.datetime.utcnow().date(),x).years))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


504407


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
median_edad = genero_edad_con_edad['edades'].median()
median_edad

29.0

In [13]:
genero_edad_sin_edad['edades'] = median_edad
genero_edad_comb_sin_fecha['edades'] = median_edad

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
genero_edad_comb = genero_edad_con_edad.append(genero_edad_sin_edad)
genero_edad_comb = genero_edad_comb.append(genero_edad_comb_sin_fecha)
len(genero_edad_comb)

504407

In [15]:
del genero_edad_con_edad
del genero_edad_sin_edad
del genero_edad_comb_sin_fecha

In [16]:
genero_edad_comb.sexo.value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

In [17]:
#One hot encoding para la columna sexo
genero_edad_comb = pd.get_dummies(genero_edad_comb, columns=["sexo"])
genero_edad_comb.head()

,idpostulante,fechanacimiento,edades,sexo_0.0,sexo_FEM,sexo_MASC,sexo_NO_DECLARA
4,eo2p,1981-02-16,37.0,0,0,1,0
8,1d2B,1976-02-28,42.0,0,0,1,0
14,EBO0,1973-09-11,44.0,0,1,0,0
32,a6MKW,1974-06-07,44.0,0,0,1,0
36,6MWd4,1974-12-19,43.0,0,0,1,0


In [18]:
genero_edad_comb = genero_edad_comb[['idpostulante', 'fechanacimiento', 'edades', 'sexo_FEM', 'sexo_MASC']]

In [19]:
genero_edad_comb.head()

,idpostulante,fechanacimiento,edades,sexo_FEM,sexo_MASC
4,eo2p,1981-02-16,37.0,0,1
8,1d2B,1976-02-28,42.0,0,1
14,EBO0,1973-09-11,44.0,1,0
32,a6MKW,1974-06-07,44.0,0,1
36,6MWd4,1974-12-19,43.0,0,1


In [20]:
genero_edad_comb.to_pickle("pickles/genero_edad_comb")

In [21]:
educacion_comb.to_pickle("pickles/educacion_comb")

In [22]:
print len(genero_edad_comb), len(educacion_comb)

504407 447909


In [4]:
genero_edad_comb = pd.read_pickle('pickles/genero_edad_comb')
educacion_comb = pd.read_pickle('pickles/educacion_comb')

In [5]:
educacion_comb.isnull().any()

idpostulante          False
estudio_abandonado    False
estudio_en_curso      False
estudio_graduado      False
dtype: bool

In [6]:
usuarios = pd.merge(genero_edad_comb, educacion_comb, how='left', on='idpostulante')

In [7]:
#Falta borrar la columna fecha nacimiento
usuarios.drop('fechanacimiento', axis=1, inplace=True)

In [8]:
usuarios.fillna(value=0)

,idpostulante,edades,sexo_FEM,sexo_MASC,estudio_abandonado,estudio_en_curso,estudio_graduado
0,eo2p,37.0,0,1,4.0,0.0,5.0
1,1d2B,42.0,0,1,0.0,4.0,0.0
2,EBO0,44.0,1,0,0.0,0.0,6.0
3,a6MKW,44.0,0,1,0.0,0.0,5.0
4,6MWd4,43.0,0,1,0.0,0.0,4.0
5,6M3jr,42.0,0,1,0.0,4.0,0.0
6,8loKM,42.0,0,1,0.0,0.0,5.0
7,ZaNm1,28.0,1,0,0.0,0.0,0.0
8,ZaG9z,45.0,1,0,0.0,0.0,4.0
9,eNje9,44.0,0,1,0.0,0.0,6.0


In [9]:
educacion_comb.isnull().any()

idpostulante          False
estudio_abandonado    False
estudio_en_curso      False
estudio_graduado      False
dtype: bool

In [31]:
usuarios.head()

,idpostulante,edades,sexo_FEM,sexo_MASC,estudio_abandonado,estudio_en_curso,estudio_graduado
0,eo2p,37.0,0,1,4.0,0.0,5.0
1,1d2B,42.0,0,1,0.0,4.0,0.0
2,EBO0,44.0,1,0,0.0,0.0,6.0
3,a6MKW,44.0,0,1,0.0,0.0,5.0
4,6MWd4,43.0,0,1,0.0,0.0,4.0


In [32]:
usuarios.to_pickle("pickles/usuarios")

## Df Visitas

In [138]:
visitas = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_3_vistas.csv"), low_memory=False)
visitasE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_3_vistas.csv"), low_memory=False)
visitas15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_3_vistas.csv"), low_memory=False)

visitas_comb = visitas.append(visitasE6)
visitas_comb = visitas_comb.append(visitas15A)

In [139]:
visitas_comb.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [ ]:
#Me interesa saber cuantas visitas realizo el usuario a cada aviso
visitas_comb = visitas_comb[["idAviso","idpostulante"]]
visitas_comb = visitas_comb.groupby(by=["idAviso","idpostulante"]).agg("sum")

In [ ]:
df_postulaciones.to_pickle('pickles/visitas')

## Df Postulaciones

In [4]:
postulaciones = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_4_postulaciones.csv"), low_memory=False)
postulacionesE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_4_postulaciones.csv"), low_memory=False)

postulaciones_comb = postulaciones.append(postulacionesE6)
len(postulaciones_comb)

8311264

In [5]:
del postulaciones
del postulacionesE6

In [6]:
postulaciones_comb = postulaciones_comb.drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')

In [7]:
len(postulaciones_comb)

6603752

In [8]:
# postulaciones_comb['fecha'] = pd.to_datetime(postulaciones_comb['fechapostulacion'], format='%Y-%m-%d %H:%M', errors='coerce')
# postulaciones_comb = postulaciones_comb[postulaciones_comb.fecha.notnull()]

In [9]:
len(postulaciones_comb)

6603752

In [10]:
#Elimino la fecha
postulaciones_comb.drop('fechapostulacion', axis=1, inplace=True)
# postulaciones_comb.drop('fecha', axis=1, inplace=True)

In [12]:
postulaciones_comb.to_pickle('pickles/postulaciones_positivas')

In [13]:
#Obtengo el dia, el mes y la hora en la que se realizo la postulacion
# postulaciones_comb['dia'] = postulaciones_comb['fecha'].apply(lambda x: x.day)
# postulaciones_comb['mes'] = postulaciones_comb['fecha'].apply(lambda x: x.month)
# #Todas las postulaciones son del 2018 por lo que no agregamos la columna ya que no aporta informacion
# #postulaciones_comb['anio'] = postulaciones_comb['fecha'].apply(lambda x: x.year)
# postulaciones_comb['hora'] = postulaciones_comb['fecha'].apply(lambda x: x.hour)
#En las predicciones que hay que realizar no tenemos data sobre la fecha, asi que esta informacion no sirve para nuestro modelo
postulaciones_comb['se_postulo'] = 1

In [14]:
postulaciones_comb.head(1)

,idaviso,idpostulante,se_postulo
0,1112257047,NM5M,1


In [15]:
#Genero informacion negativa para el clasificador

idavisos = np.copy(postulaciones_comb.idaviso.values)
np.random.shuffle(idavisos)

idpostulantes = np.copy(postulaciones_comb.idpostulante.values)
np.random.shuffle(idpostulantes)

# dias = np.copy(postulaciones_comb.dia.values)
# np.random.shuffle(dias)

# meses = np.copy(postulaciones_comb.mes.values)
# np.random.shuffle(meses)

# horas = np.copy(postulaciones_comb.hora.values)
# np.random.shuffle(horas)

postulaciones_fake = pd.DataFrame(np.array([idavisos, idpostulantes]).transpose(),
                         columns=["idaviso", "idpostulante"])
postulaciones_fake['se_postulo'] = 0

In [16]:
postulaciones_comb.to_pickle('pickles/postulaciones_posta')
postulaciones_fake.to_pickle('pickles/postulaciones_fake')

In [17]:
postulaciones_totales = postulaciones_comb.append(postulaciones_fake)
len(postulaciones_totales)

13207504

In [18]:
#Borro los duplicados que se puedan haber generado al armar datos negativos para entrenar los algoritmos
postulaciones_totales = postulaciones_totales.drop_duplicates(subset=['idpostulante', 'idaviso'], keep='first')
len(postulaciones_totales)

12988302

In [22]:
postulaciones_totales.to_pickle('pickles/postulaciones_totales')

## Dataset para entrenamiento

In [9]:
postulaciones = pd.read_pickle('pickles/postulaciones_positivas')
usuarios = pd.read_pickle("pickles/usuarios")
avisos = pd.read_pickle("pickles/avisos")

In [10]:
df_train = pd.merge(postulaciones, usuarios, how='inner', on='idpostulante')
df_train = pd.merge(df_train, avisos, how='inner', on='idaviso')

In [11]:
len(df_train)

6187929

In [12]:
df_train.columns

Index([u'idaviso', u'idpostulante', u'edades', u'sexo_FEM', u'sexo_MASC',
       u'estudio_abandonado', u'estudio_en_curso', u'estudio_graduado',
       u'idpais', u'titulo', u'descripcion', u'nombre_zona',
       u'tipo_de_trabajo', u'nivel_laboral', u'denominacion_empresa', u'ft_0',
       u'ft_1', u'ft_2', u'ft_3', u'ft_4', u'ft_5', u'ft_6', u'ft_7',
       u'nombre_area_bit_7', u'nombre_area_bit_6', u'nombre_area_bit_5',
       u'nombre_area_bit_4', u'nombre_area_bit_3', u'nombre_area_bit_2',
       u'nombre_area_bit_1', u'nombre_area_bit_0'],
      dtype='object')

In [13]:
# Sin codificacion binaria en el nombre area
# df_train = df_train[[u'idaviso', u'idpostulante', u'dia', u'mes', u'hora', u'se_postulo',
#                        u'estudio_en_curso', u'estudio_graduado',u'estudio_abandonado', u'edades', u'sexo_FEM',
#                        u'sexo_MASC', u'nombre_zona',
#                        u'tipo_de_trabajo', u'nivel_laboral', u'nombre_area',
#                        u'ft_0', u'ft_1', u'ft_2', u'ft_3', u'ft_4',
#                        u'ft_5', u'ft_6', u'ft_7']]

# Para postulaciones totales
# df_train = df_train[[u'idaviso', u'idpostulante',u'se_postulo',
#                      u'estudio_en_curso', u'estudio_graduado',u'estudio_abandonado', u'edades', u'sexo_FEM',
#                      u'sexo_MASC', u'nombre_zona',
#                      u'tipo_de_trabajo', u'nivel_laboral',
#                      u'ft_0', u'ft_1', u'ft_2', u'ft_3', u'ft_4',
#                      u'ft_5', u'ft_6', u'ft_7', u'nombre_area_bit_7', 
#                      u'nombre_area_bit_6', u'nombre_area_bit_5',
#                      u'nombre_area_bit_4', u'nombre_area_bit_3', u'nombre_area_bit_2',
#                      u'nombre_area_bit_1', u'nombre_area_bit_0']]

# Para postulaciones positivas
df_train = df_train[[u'idaviso', u'idpostulante',
                     u'estudio_en_curso', u'estudio_graduado', u'estudio_abandonado', u'edades', u'sexo_FEM',
                     u'sexo_MASC', u'nombre_zona',
                     u'tipo_de_trabajo', u'nivel_laboral',
                     u'ft_0', u'ft_1', u'ft_2', u'ft_3', u'ft_4',
                     u'ft_5', u'ft_6', u'ft_7', u'nombre_area_bit_7', 
                     u'nombre_area_bit_6', u'nombre_area_bit_5',
                     u'nombre_area_bit_4', u'nombre_area_bit_3', u'nombre_area_bit_2',
                     u'nombre_area_bit_1', u'nombre_area_bit_0']]

## One hot encoding

In [14]:
df_train = pd.get_dummies(df_train, columns=['estudio_en_curso', 'estudio_graduado', 'estudio_abandonado',
                                             'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral'])

In [15]:
df_train.columns

Index([u'idaviso', u'idpostulante', u'edades', u'sexo_FEM', u'sexo_MASC',
       u'ft_0', u'ft_1', u'ft_2', u'ft_3', u'ft_4', u'ft_5', u'ft_6', u'ft_7',
       u'nombre_area_bit_7', u'nombre_area_bit_6', u'nombre_area_bit_5',
       u'nombre_area_bit_4', u'nombre_area_bit_3', u'nombre_area_bit_2',
       u'nombre_area_bit_1', u'nombre_area_bit_0', u'estudio_en_curso_0.0',
       u'estudio_en_curso_1.0', u'estudio_en_curso_2.0',
       u'estudio_en_curso_3.0', u'estudio_en_curso_4.0',
       u'estudio_en_curso_5.0', u'estudio_en_curso_6.0',
       u'estudio_en_curso_7.0', u'estudio_graduado_0.0',
       u'estudio_graduado_1.0', u'estudio_graduado_2.0',
       u'estudio_graduado_3.0', u'estudio_graduado_4.0',
       u'estudio_graduado_5.0', u'estudio_graduado_6.0',
       u'estudio_graduado_7.0', u'estudio_abandonado_0.0',
       u'estudio_abandonado_1.0', u'estudio_abandonado_2.0',
       u'estudio_abandonado_3.0', u'estudio_abandonado_4.0',
       u'estudio_abandonado_5.0', u'estudio_a

In [16]:
if (True):
    std_edad = df_train['edades'].std()
#     std_mes = df_train['mes'].std()
#     std_dia = df_train['dia'].std()
#     std_hora = df_train['hora'].std()
    
    mean_edad = df_train['edades'].mean()
#     mean_mes = df_train['mes'].mean()
#     mean_dia = df_train['dia'].mean()
#     mean_hora = df_train['hora'].mean()
    
    df_train['edades'] = df_train['edades'].apply(lambda x: (x - mean_edad)/std_edad)
#     df_train['mes'] = df_train['mes'].apply(lambda x: (x - mean_mes)/std_mes)
#     df_train['dia'] = df_train['dia'].apply(lambda x: (x - mean_dia)/std_dia)
#     df_train['hora'] = df_train['hora'].apply(lambda x: (x - mean_hora)/std_hora)

In [17]:
df_train.isnull().any().value_counts()

False    64
dtype: int64

In [18]:
# df_train.to_pickle('pickles/df_train_totales_ohe')
df_train.to_pickle('pickles/df_train_positivos_ohe')

# Algoritmos

## One class SVM

In [4]:
df_train = pd.read_pickle('pickles/df_train_positivos_ohe')

In [6]:
from sklearn.model_selection import train_test_split  
train_data, test_data, = train_test_split(df_train, train_size = 0.9)  

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
del df_train

In [8]:
print len(train_data), "Train"
print len(test_data), "Test"

5569136 Train
618793 Test


In [9]:
from sklearn import svm

In [ ]:
#Es O(n**2) libsvm, por lo que probamos varios modelos chiquitos

In [10]:
samples = []
clfs = []
for i in range(100):
    train_data_sam = train_data.sample(10000)
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1, verbose=True)
    samples.append(train_data_sam)
    clfs.append(clf)

In [12]:
import time
t = time.time()
for i in range(100):
    clfs[i].fit(samples[i][[x for x in samples[i].columns if x not in set(["idaviso","idpostulante"])]])
elapsed = time.time() - t
print elapsed

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]185.561057091


In [15]:
from sklearn.calibration import CalibratedClassifierCV

In [16]:
# clfs_proba = []
# samples_proba = []

# t = time.time()

# for i in range(100):
#     clf_proba = CalibratedClassifierCV(svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1, verbose=True), 
#                                        method='sigmoid')
#     train_data_sam = train_data.sample(10000)
#     samples_proba.append(train_data_sam)
#     clf_proba.fit(train_data_sam[[x for x in train_data_sam.columns if x not in set(["idaviso","idpostulante"])]])
#     clfs_proba.append(clf_proba)
    
# elapsed = time.time() - t
# print elapsed

TypeError: fit() takes at least 3 arguments (2 given)

In [19]:
predictions = []
for model in clfs:
    predictions.append(model.predict(test_data[[x for x in samples[i].columns if x not in set(["idaviso","idpostulante"])]]))

## RF

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)

In [21]:
rf = RandomForestClassifier(max_depth=8, n_estimators=50, random_state=117,n_jobs=-1)

In [23]:
rf.fit(df_train[[x for x in df_train.columns if x not in set(["se_postulo","idaviso","idpostulante"])]].values, df_train['se_postulo'].values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=117, verbose=0, warm_start=False)

In [27]:
predictions = rf.predict_proba(df_train[[x for x in df_train.columns if x not in set(["se_postulo","idaviso","idpostulante"])]].values)
# predictions = predictions[:,1]
# roc_auc_score(y_score=predictions,y_true=df_train['se_postulo'].values)

In [28]:
predictions

array([[ 0.46742606,  0.53257394],
       [ 0.44373826,  0.55626174],
       [ 0.46730342,  0.53269658],
       ..., 
       [ 0.50828227,  0.49171773],
       [ 0.49060779,  0.50939221],
       [ 0.53668794,  0.46331206]])

In [29]:
import pickle
pickle.dump(rf, open("randomForestModel_ohe_y_bc", 'wb'))

## Generar submit

In [25]:
test_final = pd.read_csv(os.path.join(path_working_directory, "test_final_100k.csv"), low_memory=False)

In [31]:
m = pd.merge(test_final, avisos, on='idaviso', how='inner')
len(m)

100000

In [29]:
usuarios = pd.read_pickle("pickles/usuarios")
avisos = pd.read_pickle("pickles/avisos")

avisos = pd.get_dummies(avisos, columns=['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral'])

In [37]:
test_final = pd.merge(test_final, usuarios, how='inner', on='idpostulante')
test_final = pd.merge(test_final, avisos, how='inner', on='idaviso')

In [38]:
test_final = pd.get_dummies(test_final, columns=['estudio_en_curso', 'estudio_graduado', 'estudio_abandonado'])

In [39]:
len(test_final)

100000

In [41]:
test_final.head(1)

,id,idaviso,idpostulante,edades,sexo_FEM,sexo_MASC,idpais,titulo,descripcion,denominacion_empresa,...,tipo_de_trabajo_Por Contrato,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,0,739260,6M9ZQR,42.0,1.0,0.0,1,"[asistente, comercial]","[cliente, empresa, nac, ubicada, ciudad, bs, a...",BUMERAN.COM ARGENTINA,...,0,0,0,0,0,0,1,0,0,0


In [40]:
if (True):
    std_edad = test_final['edades'].std()
    mean_edad = test_final['edades'].mean()
    df_train['edades'] = test_final['edades'].apply(lambda x: (x - mean_edad)/std_edad)

In [ ]:
predictions = rf.predict_proba(df_train[[x for x in df_train.columns if x not in set(["id","idaviso","idpostulante"])]].values)